In [2]:
import configparser
from smartapi import SmartConnect
import pandas as pd
from datetime import datetime
from datetime import *
import requests


config = configparser.ConfigParser()

#read credentials for angel login
config.read('my_credentials.ini')

#storing in credential in a variable
client_id = config['angel']['client_id']
password = config['angel']['password']
api_key = config['angel']['api_key']

def intializeSymbolTokenMap():
    url = 'https://margincalculator.angelbroking.com/OpenAPI_File/files/OpenAPIScripMaster.json'
    d = requests.get(url).json()
    global token_df
    token_df = 'test inside fun'
    token_df = pd.DataFrame.from_dict(d)
    token_df['expiry'] = pd.to_datetime(token_df['expiry'])
    token_df = token_df.astype({'strike': float})

# fetch token/instrument from dataframe
def getTokenInfo (symbol,exch_seg='NSE',instrumenttype='OPTIDX',strike_price='',pe_ce='CE',expiry=None):
    df = token_df
    strike_price = strike_price*100
    if exch_seg == 'NSE':
        eq_df = df[(df['exch_seg'] == 'NSE') ]
        return eq_df[eq_df['name'] == symbol]
    elif exch_seg == 'NFO' and ((instrumenttype == 'FUTSTK') or (instrumenttype == 'FUTIDX')):
        return df[(df['exch_seg'] == 'NFO') & (df['instrumenttype'] == instrumenttype) & (df['name'] == symbol)].sort_values(by=['expiry'])
    elif exch_seg == 'NFO' and (instrumenttype == 'OPTSTK' or instrumenttype == 'OPTIDX'):
        return df[(df['exch_seg'] == 'NFO') & (df['instrumenttype'] == instrumenttype) & (df['name'] == symbol) & (df['strike'] == strike_price) & (df['symbol'].str.endswith(pe_ce))].sort_values(by=['expiry'])


if __name__ == '__main__':
    intializeSymbolTokenMap()
    obj=SmartConnect(api_key=api_key)
    data = obj.generateSession(client_id,password)
    refreshToken= data['data']['refreshToken']
    feedToken=obj.getfeedToken()
    # credentials.FEED_TOKEN = feedToken
    print(feedToken)
    userProfile= obj.getProfile(refreshToken)
    print(userProfile)

090105120
{'status': True, 'message': 'SUCCESS', 'errorcode': '', 'data': {'clientcode': 'A471478', 'name': 'ABDULLA NISHAD', 'email': '', 'mobileno': '', 'exchanges': ['bse_cm', 'nse_cm', 'nse_fo'], 'products': ['BO', 'NRML', 'CO', 'CNC', 'MIS', 'MARGIN'], 'lastlogintime': '', 'broker': ''}}


In [5]:
ls = []
base = 16000
for i in range(40):
    base += 50
    ls.append(base)

for i in ls:
    x = getTokenInfo('NIFTY','NFO','OPTIDX',i,'CE','2022-09-16').iloc[0]['token']
    print(f'{i} : {x}')
    

16050 : 52498
16100 : 52504
16150 : 52506
16200 : 52512
16250 : 52520
16300 : 52524
16350 : 52526
16400 : 52528
16450 : 52536
16500 : 52542
16550 : 52544
16600 : 52546
16650 : 52548
16700 : 52554
16750 : 52556
16800 : 52558
16850 : 52560
16900 : 52582
16950 : 52588
17000 : 52590
17050 : 52606
17100 : 52608
17150 : 52610
17200 : 52626
17250 : 52634
17300 : 52644
17350 : 52650
17400 : 52652
17450 : 52654
17500 : 52670
17550 : 52676
17600 : 52678
17650 : 52688
17700 : 52694
17750 : 52696
17800 : 52706
17850 : 52708
17900 : 52710
17950 : 52712
18000 : 52738


In [9]:
getTokenInfo('BANKNIFTY','NFO','OPTIDX',41100,'PE','2022-09-22').iloc[0]['token']

'59053'

In [10]:
historicParam={
            "exchange": 'NFO',
            "symboltoken": 59053 ,
            "interval": "ONE_MINUTE",
            "fromdate": "2022-09-16 09:10", 
            "todate": "2022-09-16 15:35",
            }
df = pd.DataFrame(obj.getCandleData(historicParam)['data'],columns=['date and time','open','high','low','close','volume'])
df

if df.empty:
    pass
else:
    df.to_csv(r'2022 bnf optidx\September\Expiry 22nd September\BANKNIFTYWK41100PE.csv',index=False)

path = pd.read_csv("2022 bnf optidx\September\Expiry 22nd September\BANKNIFTYWK41100PE.csv")

path



,date and time,open,high,low,close,volume
0,2022-09-16T09:15:00+05:30,522.00,570.10,486.80,555.15,7190
1,2022-09-16T09:16:00+05:30,555.70,559.90,515.35,559.30,5931
2,2022-09-16T09:17:00+05:30,565.25,603.00,565.25,573.65,7052
3,2022-09-16T09:18:00+05:30,575.35,586.85,513.40,519.25,5030
4,2022-09-16T09:19:00+05:30,512.95,514.65,491.55,503.25,6487
...,...,...,...,...,...,...
370,2022-09-16T15:25:00+05:30,614.35,614.35,594.55,603.00,1195
371,2022-09-16T15:26:00+05:30,609.15,639.80,609.15,636.75,905
372,2022-09-16T15:27:00+05:30,635.70,639.85,618.10,631.65,716
373,2022-09-16T15:28:00+05:30,632.00,638.00,629.70,634.00,480
